# Netflix data analyzation
Ευάγγελος Δημητριάδης
1115201700287

# Initialisation :

Give access to google drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Data file locations (Change to the correct path)

In [ ]:
netflix_titles = './gdrive/MyDrive/Data/netflix_titles.csv'
imdb_ratings = './gdrive/MyDrive/Data/IMDb ratings.csv'
imdb_movies = './gdrive/MyDrive/Data/IMDb movies.csv'

Scan netflix_titles for missing data.
Delete entries with missing data.

In [ ]:
import csv

lines = list()

with open(netflix_titles, 'r') as csvfile:
    csvreader = csv.reader(csvfile)
    for row in csvreader:
        if all(v for v in row):
            lines.append(row)

with open(netflix_titles, 'w') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(lines)

Clean up IMDb Movies file to avoid errors

In [ ]:
with open(imdb_movies, 'r') as csvfile:
    csvreader = csv.reader(csvfile)
    for row in csvreader:
      for i, x in enumerate(row):
                  if len(x)< 1:
                          x = row[i] = "N/A"

# Queries :

Import libraries

In [ ]:
import csv
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from dateutil import parser

Pandas options

In [ ]:
pd.set_option('display.max_rows', None)

**1.** Are there more movies or TV shows on netflix?

In [ ]:
df = pd.read_csv(netflix_titles)
num_series=df['type'].value_counts().get('TV Show',0)
num_movies=df['type'].value_counts().get('Movie',0)
print("Series:",num_series)
print("Movies:",num_movies)
if(num_series>num_movies):
    print("There are more TV shows on Netflix")
else:
    print("There are more movies on Netflix")

**2.** Has Netflix invested more on movies or TV shows in the past few years ?

In [ ]:
now = datetime.datetime.now()
current_year = now.year
recent_years = 3

num_series=df[df['release_year']>=current_year-recent_years]['type'].value_counts().get('TV Show',0)
num_movies=df[df['release_year']>=current_year-recent_years]['type'].value_counts().get('Movie',0)
print(current_year-recent_years,"-",current_year)
print("Series:",num_series)
print("Movies:",num_movies)
if(num_series>num_movies):
    print("Netflix invests more in TV shows")
else:
    print("Netflix invests more in movies")

**3.** Country with the most content



In [ ]:
df = pd.read_csv(netflix_titles)
country = df['country'].value_counts().keys().tolist()[0]
titles = df['country'].value_counts().tolist()[0]
print(country,":",titles,"titles")

**4.** Genres produced by each country



In [ ]:
df = pd.read_csv(netflix_titles)
countries=df['country'].str.split(',', expand=True).stack().unique()
countries.sort()
countries = list(filter(None, countries))
for country in countries:
    genres=df[df['country'].str.contains(country)]['listed_in'].str.split(',', expand=True).stack().unique()
    genres.sort()
    gnr=','.join(genres)
    print(country,":",gnr,"\n")

**5.** Actors with the most appearances in movies and TV shows for each country.

In [ ]:
#Movies
df = pd.read_csv(netflix_titles)
countries=df['country'].str.split(',', expand=True).stack().unique()
countries.sort()
countries = list(filter(None, countries))
for country in countries:
    df2=df[df['country'].str.contains(country)]
    actors = df2[df2['type']=='Movie']['cast']
    appearances = df2[df2['type']=='Movie']['cast']
    if(len(actors)==0 or len(appearances)==0):
        continue
    actors=actors.str.split(',',expand=True).stack().value_counts().keys().tolist()
    appearances = appearances.str.split(',',expand=True).stack().value_counts().tolist()

    data={'Actor':actors,'No. Appearances':appearances}
    ap=pd.DataFrame(data)
    ap.index = range(1, len(ap)+1)
    print(country)
    display(ap[:10])

In [ ]:
#TV Shows
df = pd.read_csv(netflix_titles)
countries=df['country'].str.split(',', expand=True).stack().unique()
countries.sort()
countries = list(filter(None, countries))
for country in countries:
    df2=df[df['country'].str.contains(country)]
    actors = df2[df2['type']=='TV Show']['cast']
    appearances = df2[df2['type']=='TV Show']['cast']
    if(len(actors)==0 or len(appearances)==0):
        continue
    actors=actors.str.split(',',expand=True).stack().value_counts().keys().tolist()
    appearances = appearances.str.split(',',expand=True).stack().value_counts().tolist()

    data={'Actor':actors,'No. Appearances':appearances}
    ap=pd.DataFrame(data)
    ap.index = range(1, len(ap)+1)
    print(country)
    display(ap[:10])

**6.** Distribution of age appropriate content

In [ ]:
#Movies
df = pd.read_csv(netflix_titles)
counts=df[df['type']=='Movie']['rating'].value_counts()
lk=sum([counts.get(k,0) for k in ['G','TV-Y','TV-G']])
ok=sum([counts.get(k,0) for k in ['PG','TV-Y7','TV-Y7-FV','TV-PG']])
t=sum([counts.get(k,0) for k in ['PG-13','TV-14']])
m=sum([counts.get(k,0) for k in ['R','NC-17','TV-MA']])
data = [[lk,ok,t,m]]
ratings = pd.DataFrame(data, columns = ['Little Kids','Older Kids','Teens','Mature'])
ratings.index = ['']
print("Movies")
ratings

In [ ]:
#TV Shows
df = pd.read_csv(netflix_titles)
counts=df[df['type']=='TV Show']['rating'].value_counts()
lk=sum([counts.get(k,0) for k in ['G','TV-Y','TV-G']])
ok=sum([counts.get(k,0) for k in ['PG','TV-Y7','TV-Y7-FV','TV-PG']])
t=sum([counts.get(k,0) for k in ['PG-13','TV-14']])
m=sum([counts.get(k,0) for k in ['R','NC-17','TV-MA']])
data = [[lk,ok,t,m]]
ratings = pd.DataFrame(data, columns = ['Little Kids','Older Kids','Teens','Mature'])
ratings.index = ['']
print("TV Shows")
ratings

**7.** Average Netflix releases per month

In [ ]:
df = pd.read_csv(netflix_titles)
years=df['date_added'].tolist()
for i in range(len(years)):
    years[i]=parser.parse(years[i])
    years[i]=years[i].year
years = list(set(years))
years.sort()

months = ["January","February","March",
          "April","May","June","July",
          "August","September","October",
          "November","December"]

entries={}
for month in months:
    entries[month] = 0

for month in months:
    count=df[df['date_added'].str.contains(month)]
    entries[month]+=len(count)
    entries[month]=entries[month]/len(years)

print(years[0],"-",years[-1])
print("Average Netflix releases per month")
avg=pd.DataFrame.from_dict(entries,orient='index')
avg.columns = ['Average Releases']
avg.index.name = 'Month'
avg.plot.bar()
avg


**8.** All genres available on netflix

In [ ]:
df = pd.read_csv(netflix_titles)
genres=df['listed_in'].str.split(',', expand=True).stack().unique()
genres.sort()

gnr=pd.DataFrame(genres,columns=["Genre"])
gnr=gnr.style.hide_index()
gnr

**9.** Directors analyzed by country

In [ ]:
df = pd.read_csv(netflix_titles)
countries=df['country'].str.split(',', expand=True).stack().unique().tolist()
countries.sort()
countries = list(filter(None, countries))

In [ ]:
#Number of directors per country
counts = {}
for country in countries:
    counts[country]=len(df[df['country'].str.contains(country)]['director'].str.split(',', expand=True).stack().unique())
dpc=pd.DataFrame.from_dict(counts,orient='index')
dpc.columns = ['No. directors']
dpc.index.name = 'Country'
dpc=dpc.sort_values(by=['No. directors'], ascending=False)
dpc

In [ ]:
#Top directors per country (By number of movies)
top_dir={}
for country in countries:
    top_dir[country]=[df[df['country'].str.contains(country)]['director'].str.split(',', expand=True).stack().value_counts().keys().tolist()[0]
    ,df[df['country'].str.contains(country)]['director'].str.split(',', expand=True).stack().value_counts().tolist()[0]]
tdpc=pd.DataFrame.from_dict(top_dir,orient='index')
tdpc.columns = ['Director','No. Ttiles']
tdpc.index.name = 'Country'
tdpc

**10.** Series organized by number of seasons

In [ ]:
df = pd.read_csv(netflix_titles)
series=df[df['type']=="TV Show"]

seasons=series['duration'].unique()
seasons.sort()

entries={}
for season in seasons:
    entries[season]=[(','.join(series[series['duration']==season]['title'])),len(series[series['duration']==season]['title'])]


sps=pd.DataFrame.from_dict(entries,orient='index')
sps.columns = ['Series Titles','No. Shows']
sps.index=(int(s.split(' ',1)[0]) for s in sps.index)
sps=sps.sort_index()
sps.index.name = 'No. Seasons'
sps

**11.** Top rated movies on Netflix based on IMDb ratings



In [ ]:
#11
df1 = pd.read_csv(netflix_titles)
df1= df1[df1['type']=='Movie']
df2 = pd.read_csv(imdb_movies,dtype=object)
df3 = pd.read_csv(imdb_ratings)

df=df1.merge(df2,how='inner',left_on=['title'],right_on=['title'])
df=df.merge(df3,how='inner',left_on=['imdb_title_id'],right_on=['imdb_title_id'])

df=df.sort_values(by=['weighted_average_vote'], ascending=False)
df=df[['show_id','title','weighted_average_vote']]
df.index = range(1, len(df)+1)
df[:10]

# Recommendation system :

Import libraries

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import cdist
import numpy as np
import re

Dataframes used:

In [ ]:
df = pd.read_csv(netflix_titles)
movies=df[df['type']=='Movie']

import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
movies['description']=movies['description'].replace('[^a-zA-Z ]','',regex=True).astype(str)#Strip all special characters and digits

ids=movies['show_id']

### **1.**

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self,text):#Lemmatize and remove stop_words
        return [self.wnl.lemmatize(t) for t in word_tokenize(text) if t not in stopwords.words('english')]

**a**.) Create boolean BoW(Bag of Words) table for unigrams and bigrams of Netflix movie descriptions

In [ ]:
bow_v = CountVectorizer(ngram_range=(1, 2),binary=True,tokenizer=LemmaTokenizer(),max_df=1.0, min_df=1)
bow_x = bow_v.fit_transform(movies['description'])

bow_words=bow_v.get_feature_names()
bow_list=bow_x.toarray()

bow = {}
i=0
for id in ids:
        bow[id] = bow_list[i]
        i+=1

b.)Create TF-IDF (Term Frequency - InverseDocument Frequency) table for unigrams and bigrams of Netflix movie descriptions

In [ ]:
tfidf_v = TfidfVectorizer(ngram_range=(1, 2),tokenizer=LemmaTokenizer(),max_df=1.0, min_df=1)
tfidf_x = tfidf_v.fit_transform(movies['description'])

tfidf_words=tfidf_v.get_feature_names()
tfidf_list=tfidf_x.toarray()

tfidf = {}
i=0
for id in ids:
        tfidf[id] = tfidf_list[i]
        i+=1

### **2.**

Find similarity between two movies:

a.)Using Jaccard/Tanimoto coefficient on Boolean BoW vector

In [ ]:
def movies_similarity_bow(id1,id2):
    return 1-cdist([bow[id1]],[bow[id2]],metric='jaccard')

b.)Using cosine similarity on TF-IDF vector

In [ ]:
def movies_similarity_tfidf(id1,id2):
    return 1-cdist([tfidf[id1]],[tfidf[id2]],metric='cosine')

Find the 100 most similar movies for each movie using the two above methods (Boolean BoW Jaccard/Tanimoto
coefficient - TF-IDF  cosine similarity)

In [ ]:
# Runs slow ,can be sped up by creating a smaller vocabulary
# On 1.a and 1.b you can pass max_features=n argument on the vectorizer to achieve that

#Top-100 using boolean bow and jaccard
similar_bow={}
for id in ids:
    similar_list=1-cdist([bow[id]],bow_list,metric='jaccard')[0]
    similar = {}
    i=0
    for id in ids:
            similar[id] = similar_list[i]
            i+=1
    del similar[id]
    similar=dict(sorted(similar.items(), key=lambda item: item[1], reverse=True)[:100])
    similar_bow[id]=similar

#Top-100 using tf-idf and cosine
similar_tfidf={}
for id in ids:
    similar_list=1-cdist([tfidf[id]],tfidf_list,metric='cosine')[0]
    similar = {}
    i=0
    for id in ids:
            similar[id] = similar_list[i]
            i+=1
    del similar[id]
    similar=dict(sorted(similar.items(), key=lambda item: item[1], reverse=True)[:100])
    similar_tfidf[id]=similar

### **3.** Find n most similar movies for movie with given title using the given method

In [ ]:
def get_similar_movies1(title, n, method):
    s_id=movies[movies['title']==title]['show_id']
    s_id.index = range(0, len(s_id))
    s_id=s_id[0]

    if method=='boolean':
        similar_list=1-cdist([bow[s_id]],bow_list,metric='jaccard')[0]
    elif method=='tf-idf':
        similar_list=1-cdist([tfidf[s_id]],tfidf_list,metric='cosine')[0]

    similar = {}
    i=0
    for id in ids:
            similar[id] = similar_list[i]
            i+=1
    del similar[s_id]
    similar=sorted(similar.items(), key=lambda item: item[1], reverse=True)[:n]

    #Create pandas dataframe for results
    similar_ids,similarities=zip(*similar)
    similar_titles=[None] * len(similar_ids)
    i=0
    for id in similar_ids:
      similar_titles[i]=movies[movies['show_id']==id]['title'].iloc[0]
      i+=1
    similar_df=pd.DataFrame(
    {'show_id': similar_ids,
    'title': similar_titles,
    'similarity': similarities
    })
    similar_df.index = range(1, len(similar_df)+1)

    return similar_df

Indicative results :

Η boolean μέθοδος δεν λαμβάνει υπόψιν το πόσες φορές εμφανίζεται ένα term στο description(bow),την συχνότητα εμφάνισης(tf), ούτε την σημαντικότητα του term(idf).Μετράει απλά αν μια λέξη βρίσκεται στην πρόταση ή όχι.Έτσι φυσικά είναι λιγότερο ακριβής στον υπολογισμό του similarity από την μέθοδο που χρησιμοποιεί τον tf-idf.

In [ ]:
print("Django Unchained")
print("Boolean method:")
display(get_similar_movies1('Django Unchained', 20, 'boolean'))
print("TF-IDF method:")
display(get_similar_movies1('Django Unchained', 20, 'tf-idf'))

Περιγραφή ταινίας "Django Unchained" : Accompanied by a German bounty hunter, a freed slave named Django travels across America to free his wife from a sadistic plantation owner.

Οι δύο μέθοδοι έχουν αρκετά παρόμοια αποτελέσματα για την ταινία που δόθηκε(Western και ταινίες δράσης),η boolean μέθοδος βγάζει όμως πιο ανακριβή αποτελέσματα.
Έχει πολλές κοινές ταινίες με την tf-idf αλλά έχει χαμηλότερο similarity για ταινίες western και στα κορυφαία αποτελέσματα εμφανίζει την ταινία "Un plus une" με την οποία δεν μοιράζονται κάποιο σημαντικό term(μόνο την λέξη travel).

In [ ]:
print("LEGO Marvel Super Heroes: Avengers Reassembled!")
print("Boolean method:")
display(get_similar_movies1('LEGO Marvel Super Heroes: Avengers Reassembled!', 20, 'boolean'))
print("TF-IDF method:")
display(get_similar_movies1('LEGO Marvel Super Heroes: Avengers Reassembled!', 20, 'tf-idf'))

Περιγραφή ταινίας "LEGO Marvel Super Heroes: Avengers Reassembled!" : "When Ultron seizes control of Iron Man's armor, the Avengers race to save their friend and stop an evil plot to take over the world."
 
Στο παραπάνω παράδειγμα η απόκλιση των μεθόδων γίνεται ακόμα πιο εμφανής.Η μέθοδος tf-idf έχει τις ταινίες marvel ως κορυφαίες προτάσεις ενώ η boolean ως χαμηλότερες.Επίσης η boolean ενώ εμφανίζει αρκετές κοινές ταινίες(παιδικές και marvel) εμφανίζει και τελείως άσχετες ταινίες όπως "Nymphomaniac: Volume 1","30 Days of Luxury" και "My Step Dad: The Hippie".

### **4.** Find n most similar movies for given description using the given method

In [ ]:
def get_similar_movies2(description,n,method):
    #Create new dataframe for description
    title = pd.DataFrame([['search_item',description]],columns=['show_id','description'])
    s_id='search_item'
    title['description']=title['description'].replace('[^a-zA-Z ]','',regex=True).astype(str)#Strip all special characters and digits

    if method=='boolean': #BoW
        #Create new feature vector from dataframe
        tbow_v = CountVectorizer(ngram_range=(1, 2),binary=True,tokenizer=LemmaTokenizer(),max_df=1.0, min_df=1)
        tbow_x = tbow_v.fit_transform(title['description'])
        tbow_words=tbow_v.get_feature_names()
        ttbow_list=tbow_x.toarray()[0]

        #Initialize array to make the comparison
        tbow_list = [0] * len(bow_words)
        for gram in tbow_words:
            if gram in bow_words:
                tbow_list[bow_words.index(gram)]=1

        #Find similarities between description and movies
        similar_list=1-cdist([tbow_list],bow_list,metric='jaccard')[0]
    elif method=='tf-idf': #Tf-idf
        #Create new feature vector from dataframe
        ttfidf_v = TfidfVectorizer(ngram_range=(1, 2),tokenizer=LemmaTokenizer())
        ttfidf_x = ttfidf_v.fit_transform(title['description'])
        ttfidf_words=ttfidf_v.get_feature_names()
        tttfidf_list=ttfidf_x.toarray()[0]

        #Initialize array to make the comparison
        ttfidf_list = [0] * len(tfidf_words)
        i=0
        for gram in ttfidf_words:
            if gram in tfidf_words:
                ttfidf_list[tfidf_words.index(gram)]=tttfidf_list[i]
            i+=1
            
        #Find similarities between description and movies
        similar_list=1-cdist([ttfidf_list],tfidf_list,metric='cosine')[0]

    #Find n most common movies for given description
    similar = {}
    i=0
    for id in ids:
            similar[id] = similar_list[i]
            i+=1
    similar=sorted(similar.items(), key=lambda item: item[1], reverse=True)[:n]

    #Create pandas dataframe for results
    similar_ids,similarities=zip(*similar)
    similar_titles=[None] * len(similar_ids)
    i=0
    for id in similar_ids:
      similar_titles[i]=movies[movies['show_id']==id]['title'].iloc[0]
      i+=1
    similar_df=pd.DataFrame(
    {'show_id': similar_ids,
    'title': similar_titles,
    'similarity': similarities
    })
    similar_df.index = range(1, len(similar_df)+1)

    return similar_df

Indicative results :

In [ ]:
description_test="Accompanied by a German bounty hunter, a freed slave named Django travels across America to free his wife from a sadistic plantation owner."

print("Django Unchained")
print("Boolean method:")
display(get_similar_movies2(description_test, 20, 'boolean'))
print("TF-IDF method:")
display(get_similar_movies2(description_test, 20, 'tf-idf'))

Η συνάρτηση δημιουργεί ένα feature vector για το description που δίνεται.
Μετά το διαμορφώνει ώστε τα terms του να είναι τα ίδια με αυτά του vocabulary(αν κάποιο term υπάρχει στο vocabulary του description αλλά όχι στων ταινιών τότε απορρίπτεται).
 
Η περιγραφή που περνιέται σαν όρισμα είναι αυτή της ταινίας "Django Unchained".
 
Η boolean μέθοδος βγάζει ακριβώς τα ίδια αποτελέσματα(και similarity 1.0 με την ταινία που έχει το ίδιο description), αφού ο boolean BoW πίνακας δείχνει απλά αν ένα term υπάρχει στο description ή όχι οπότε δεν υπάρχει κάποια διαφορά με το να φτιαχτεί με τα υπόλοιπα descriptions η ξεχωριστά(αν όλα τα terms του description υπάρχουν στο vocabulary).
 
Η tf-idf μέθοδος έχει μια απόκλιση αφού ο πίνακας δημιουργήθηκε μόνο για αυτό το description (ξεχωριστά από τα descriptions), οπότε δεν μπορεί να υπολογιστεί σωστά το idf. Έτσι υπάρχουν τίτλοι διαφορετικοί,με διαφορετικό similarity score η σε διαφορετική σειρά.

